In [2]:
import os, glob, math, numpy as np, torch
import torchaudio
from tqdm import tqdm
from transformers import AutoFeatureExtractor, HubertModel

In [3]:
AUDIO_DIR   = "/mnt/d/Roshidat_Msc_Project/Audio_parkinson/MDVR-KCL_Dataset/26_29_09_2017_KCL/26-29_09_2017_KCL/ReadText/PD"     # folder with .wav/.flac/.mp3...
OUT_DIR     = os.path.join("/mnt/d/Roshidat_Msc_Project/Audio_parkinson/pd&Hc_multi/Hubert embeddings", "PD_ReadText_hubert_features")
MODEL_NAME  = "facebook/hubert-base-ls960"    # or "facebook/hubert-large-ll60k"
DEVICE      = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TARGET_SR   = 16000
CHUNK_SEC   = 30.0        # process long files in ~30s chunks to avoid OOM
STRIDE_SEC  = 30.0        # == CHUNK_SEC -> no overlap; keep simple concatenation
SAVE_DTYPE  = np.float32  # set to np.float16 to halve disk space

AUDIO_EXTS  = (".wav", ".flac", ".mp3", ".m4a", ".ogg")

os.makedirs(OUT_DIR, exist_ok=True)

In [4]:
feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)
model = HubertModel.from_pretrained(MODEL_NAME).to(DEVICE).eval()
for p in model.parameters(): p.requires_grad = False

def load_audio(path, target_sr=TARGET_SR):
    wav, sr = torchaudio.load(path)              # [C,T]
    if wav.size(0) > 1: wav = wav.mean(0, keepdim=True)  # mono
    if sr != target_sr: wav = torchaudio.functional.resample(wav, sr, target_sr)
    return wav.squeeze(0), target_sr             # [T], 16000

def iter_chunks(wav, sr, chunk_sec=CHUNK_SEC, stride_sec=STRIDE_SEC):
    T = wav.numel()
    if chunk_sec <= 0 or T <= int(chunk_sec*sr):
        yield wav; return
    L = int(chunk_sec*sr); S = int(stride_sec*sr)
    i = 0
    while i < T:
        j = min(i+L, T)
        yield wav[i:j]
        if j == T: break
        i += S

@torch.no_grad()
def hubert_embed_1d(wav_1d, sr=TARGET_SR):
    inputs = feature_extractor(wav_1d.numpy(), sampling_rate=sr, return_tensors="pt")
    out = model(**{k: v.to(DEVICE) for k, v in inputs.items()})
    feats = out.last_hidden_state[0]                 # [L, D]
    mask  = torch.ones(feats.size(0), dtype=torch.bool)
    return feats.cpu(), mask

def process_file(path):
    base = os.path.splitext(os.path.basename(path))[0]
    out_path = os.path.join(OUT_DIR, f"{base}_hubert_feats.npz")
    if os.path.isfile(out_path): return

    wav, sr = load_audio(path, TARGET_SR)
    feats_all, masks_all = [], []
    for chunk in iter_chunks(wav, sr):
        f, m = hubert_embed_1d(chunk, sr)
        feats_all.append(f); masks_all.append(m)

    feats = torch.cat(feats_all, 0).numpy().astype(SAVE_DTYPE)  # [L, D]
    mask  = torch.cat(masks_all, 0).numpy()                     # [L]
    dur   = wav.numel()/sr
    frame_hz = feats.shape[0]/max(dur, 1e-9)

    np.savez_compressed(
        out_path,
        hubert_embeddings=feats,
        attention_mask=mask,
        sample_rate=np.array(sr),
        model=np.array(MODEL_NAME),
        frame_hz=np.array(frame_hz, dtype=np.float32),
        audio_duration_sec=np.array(dur, dtype=np.float32),
        chunk_sec=np.array(CHUNK_SEC, dtype=np.float32),
        stride_sec=np.array(STRIDE_SEC, dtype=np.float32),
        source_audio=os.path.basename(path),
    )
    print(f"[ok] {os.path.basename(path)} -> {feats.shape}")

def main():
    files = [p for p in glob.glob(os.path.join(AUDIO_DIR, "**", "*"), recursive=True)
             if os.path.splitext(p)[1].lower() in AUDIO_EXTS]
    files.sort()
    print(f"Found {len(files)} audio files.")
    for p in tqdm(files, desc="HuBERT"):
        try: process_file(p)
        except Exception as e: print(f"[err] {os.path.basename(p)}: {e}")

if __name__ == "__main__":
    main()


Found 16 audio files.


HuBERT:   6%|▋         | 1/16 [00:02<00:31,  2.11s/it]

[ok] ID02_pd_2_0_0.wav -> (7872, 768)


HuBERT:  12%|█▎        | 2/16 [00:03<00:22,  1.61s/it]

[ok] ID04_pd_2_0_1.wav -> (6119, 768)


...

[ok] ID32_pd_3_1_1.wav -> (3649, 768)


HuBERT:  94%|█████████▍| 15/16 [00:24<00:02,  2.47s/it]

[ok] ID33_pd_3_2_2.wav -> (5915, 768)


HuBERT: 100%|██████████| 16/16 [00:25<00:00,  1.62s/it]

[ok] ID34_pd_2_0_0.wav -> (6391, 768)


In [8]:
AUDIO_DIR   = "/mnt/d/Roshidat_Msc_Project/Audio_parkinson/MDVR-KCL_Dataset/26_29_09_2017_KCL/26-29_09_2017_KCL/ReadText/HC"     # folder with .wav/.flac/.mp3...
OUT_DIR     = os.path.join("/mnt/d/Roshidat_Msc_Project/Audio_parkinson/pd&Hc_multi/Hubert embeddings", "HC_ReadText_hubert_features")

os.makedirs(OUT_DIR)

feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)
model = HubertModel.from_pretrained(MODEL_NAME).to(DEVICE).eval()
for p in model.parameters(): p.requires_grad = False

def load_audio(path, target_sr=TARGET_SR):
    wav, sr = torchaudio.load(path)              # [C,T]
    if wav.size(0) > 1: wav = wav.mean(0, keepdim=True)  # mono
    if sr != target_sr: wav = torchaudio.functional.resample(wav, sr, target_sr)
    return wav.squeeze(0), target_sr             # [T], 16000

def iter_chunks(wav, sr, chunk_sec=CHUNK_SEC, stride_sec=STRIDE_SEC):
    T = wav.numel()
    if chunk_sec <= 0 or T <= int(chunk_sec*sr):
        yield wav; return
    L = int(chunk_sec*sr); S = int(stride_sec*sr)
    i = 0
    while i < T:
        j = min(i+L, T)
        yield wav[i:j]
        if j == T: break
        i += S

@torch.no_grad()
def hubert_embed_1d(wav_1d, sr=TARGET_SR):
    inputs = feature_extractor(wav_1d.numpy(), sampling_rate=sr, return_tensors="pt")
    out = model(**{k: v.to(DEVICE) for k, v in inputs.items()})
    feats = out.last_hidden_state[0]                 # [L, D]
    mask  = torch.ones(feats.size(0), dtype=torch.bool)
    return feats.cpu(), mask

def process_file(path):
    base = os.path.splitext(os.path.basename(path))[0]
    out_path = os.path.join(OUT_DIR, f"{base}_hubert_feats.npz")
    if os.path.isfile(out_path): return

    wav, sr = load_audio(path, TARGET_SR)
    feats_all, masks_all = [], []
    for chunk in iter_chunks(wav, sr):
        f, m = hubert_embed_1d(chunk, sr)
        feats_all.append(f); masks_all.append(m)

    feats = torch.cat(feats_all, 0).numpy().astype(SAVE_DTYPE)  # [L, D]
    mask  = torch.cat(masks_all, 0).numpy()                     # [L]
    dur   = wav.numel()/sr
    frame_hz = feats.shape[0]/max(dur, 1e-9)

    np.savez_compressed(
        out_path,
        hubert_embeddings=feats,
        attention_mask=mask,
        sample_rate=np.array(sr),
        model=np.array(MODEL_NAME),
        frame_hz=np.array(frame_hz, dtype=np.float32),
        audio_duration_sec=np.array(dur, dtype=np.float32),
        chunk_sec=np.array(CHUNK_SEC, dtype=np.float32),
        stride_sec=np.array(STRIDE_SEC, dtype=np.float32),
        source_audio=os.path.basename(path),
    )
    print(f"[ok] {os.path.basename(path)} -> {feats.shape}")

def main():
    files = [p for p in glob.glob(os.path.join(AUDIO_DIR, "**", "*"), recursive=True)
             if os.path.splitext(p)[1].lower() in AUDIO_EXTS]
    files.sort()
    print(f"Found {len(files)} audio files.")
    for p in tqdm(files, desc="HuBERT"):
        try: process_file(p)
        except Exception as e: print(f"[err] {os.path.basename(p)}: {e}")

if __name__ == "__main__":
    main()


Found 21 audio files.


HuBERT:   5%|▍         | 1/21 [00:02<00:44,  2.22s/it]

[ok] ID00_hc_0_0_0.wav -> (7550, 768)


HuBERT:  10%|▉         | 2/21 [00:03<00:36,  1.90s/it]

[ok] ID01_hc_0_0_0.wav -> (8200, 768)


...

[ok] ID31_hc_0_1_1.wav -> (9642, 768)


HuBERT:  95%|█████████▌| 20/21 [00:32<00:01,  1.63s/it]

[ok] ID35_hc_0_0_0.wav -> (4658, 768)


HuBERT: 100%|██████████| 21/21 [00:34<00:00,  1.64s/it]

[ok] ID36_hc_0_0_0.wav -> (7924, 768)


In [5]:
AUDIO_DIR   = "/mnt/d/Roshidat_Msc_Project/Audio_parkinson/MDVR-KCL_Dataset/26_29_09_2017_KCL/26-29_09_2017_KCL/SpontaneousDialogue/HC"     # folder with .wav/.flac/.mp3...
OUT_DIR     = os.path.join("/mnt/d/Roshidat_Msc_Project/Audio_parkinson/pd&Hc_multi/HC_Spontaneous", "HC_Spontaneous_hubert_features")

os.makedirs(OUT_DIR)

feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)
model = HubertModel.from_pretrained(MODEL_NAME).to(DEVICE).eval()
for p in model.parameters(): p.requires_grad = False

def load_audio(path, target_sr=TARGET_SR):
    wav, sr = torchaudio.load(path)              # [C,T]
    if wav.size(0) > 1: wav = wav.mean(0, keepdim=True)  # mono
    if sr != target_sr: wav = torchaudio.functional.resample(wav, sr, target_sr)
    return wav.squeeze(0), target_sr             # [T], 16000

def iter_chunks(wav, sr, chunk_sec=CHUNK_SEC, stride_sec=STRIDE_SEC):
    T = wav.numel()
    if chunk_sec <= 0 or T <= int(chunk_sec*sr):
        yield wav; return
    L = int(chunk_sec*sr); S = int(stride_sec*sr)
    i = 0
    while i < T:
        j = min(i+L, T)
        yield wav[i:j]
        if j == T: break
        i += S

@torch.no_grad()
def hubert_embed_1d(wav_1d, sr=TARGET_SR):
    inputs = feature_extractor(wav_1d.numpy(), sampling_rate=sr, return_tensors="pt")
    out = model(**{k: v.to(DEVICE) for k, v in inputs.items()})
    feats = out.last_hidden_state[0]                 # [L, D]
    mask  = torch.ones(feats.size(0), dtype=torch.bool)
    return feats.cpu(), mask

def process_file(path):
    base = os.path.splitext(os.path.basename(path))[0]
    out_path = os.path.join(OUT_DIR, f"{base}_hubert_feats.npz")
    if os.path.isfile(out_path): return

    wav, sr = load_audio(path, TARGET_SR)
    feats_all, masks_all = [], []
    for chunk in iter_chunks(wav, sr):
        f, m = hubert_embed_1d(chunk, sr)
        feats_all.append(f); masks_all.append(m)

    feats = torch.cat(feats_all, 0).numpy().astype(SAVE_DTYPE)  # [L, D]
    mask  = torch.cat(masks_all, 0).numpy()                     # [L]
    dur   = wav.numel()/sr
    frame_hz = feats.shape[0]/max(dur, 1e-9)

    np.savez_compressed(
        out_path,
        hubert_embeddings=feats,
        attention_mask=mask,
        sample_rate=np.array(sr),
        model=np.array(MODEL_NAME),
        frame_hz=np.array(frame_hz, dtype=np.float32),
        audio_duration_sec=np.array(dur, dtype=np.float32),
        chunk_sec=np.array(CHUNK_SEC, dtype=np.float32),
        stride_sec=np.array(STRIDE_SEC, dtype=np.float32),
        source_audio=os.path.basename(path),
    )
    print(f"[ok] {os.path.basename(path)} -> {feats.shape}")

def main():
    files = [p for p in glob.glob(os.path.join(AUDIO_DIR, "**", "*"), recursive=True)
             if os.path.splitext(p)[1].lower() in AUDIO_EXTS]
    files.sort()
    print(f"Found {len(files)} audio files.")
    for p in tqdm(files, desc="HuBERT"):
        try: process_file(p)
        except Exception as e: print(f"[err] {os.path.basename(p)}: {e}")

if __name__ == "__main__":
    main()


Found 21 audio files.


HuBERT:   5%|▍         | 1/21 [00:01<00:26,  1.34s/it]

[ok] ID00_hc_0_0_0s.wav -> (5988, 768)


HuBERT:  10%|▉         | 2/21 [00:02<00:22,  1.21s/it]

[ok] ID01_hc_0_0_0s.wav -> (5850, 768)


...

[ok] ID31_hc_0_1_1s.wav -> (3815, 768)


HuBERT:  95%|█████████▌| 20/21 [00:31<00:01,  1.28s/it]

[ok] ID35_hc_0_0_0s.wav -> (7418, 768)


HuBERT: 100%|██████████| 21/21 [00:32<00:00,  1.54s/it]

[ok] ID36_hc_0_0_0s.wav -> (5665, 768)


In [6]:
AUDIO_DIR   = "/mnt/d/Roshidat_Msc_Project/Audio_parkinson/MDVR-KCL_Dataset/26_29_09_2017_KCL/26-29_09_2017_KCL/SpontaneousDialogue/PD"     # folder with .wav/.flac/.mp3...
OUT_DIR     = os.path.join("/mnt/d/Roshidat_Msc_Project/Audio_parkinson/pd&Hc_multi/PD_Spontaneous", "PD_Spontaneous_hubert_features")

os.makedirs(OUT_DIR)

feature_extractor = AutoFeatureExtractor.from_pretrained(MODEL_NAME)
model = HubertModel.from_pretrained(MODEL_NAME).to(DEVICE).eval()
for p in model.parameters(): p.requires_grad = False

def load_audio(path, target_sr=TARGET_SR):
    wav, sr = torchaudio.load(path)              # [C,T]
    if wav.size(0) > 1: wav = wav.mean(0, keepdim=True)  # mono
    if sr != target_sr: wav = torchaudio.functional.resample(wav, sr, target_sr)
    return wav.squeeze(0), target_sr             # [T], 16000

def iter_chunks(wav, sr, chunk_sec=CHUNK_SEC, stride_sec=STRIDE_SEC):
    T = wav.numel()
    if chunk_sec <= 0 or T <= int(chunk_sec*sr):
        yield wav; return
    L = int(chunk_sec*sr); S = int(stride_sec*sr)
    i = 0
    while i < T:
        j = min(i+L, T)
        yield wav[i:j]
        if j == T: break
        i += S

@torch.no_grad()
def hubert_embed_1d(wav_1d, sr=TARGET_SR):
    inputs = feature_extractor(wav_1d.numpy(), sampling_rate=sr, return_tensors="pt")
    out = model(**{k: v.to(DEVICE) for k, v in inputs.items()})
    feats = out.last_hidden_state[0]                 # [L, D]
    mask  = torch.ones(feats.size(0), dtype=torch.bool)
    return feats.cpu(), mask

def process_file(path):
    base = os.path.splitext(os.path.basename(path))[0]
    out_path = os.path.join(OUT_DIR, f"{base}_hubert_feats.npz")
    if os.path.isfile(out_path): return

    wav, sr = load_audio(path, TARGET_SR)
    feats_all, masks_all = [], []
    for chunk in iter_chunks(wav, sr):
        f, m = hubert_embed_1d(chunk, sr)
        feats_all.append(f); masks_all.append(m)

    feats = torch.cat(feats_all, 0).numpy().astype(SAVE_DTYPE)  # [L, D]
    mask  = torch.cat(masks_all, 0).numpy()                     # [L]
    dur   = wav.numel()/sr
    frame_hz = feats.shape[0]/max(dur, 1e-9)

    np.savez_compressed(
        out_path,
        hubert_embeddings=feats,
        attention_mask=mask,
        sample_rate=np.array(sr),
        model=np.array(MODEL_NAME),
        frame_hz=np.array(frame_hz, dtype=np.float32),
        audio_duration_sec=np.array(dur, dtype=np.float32),
        chunk_sec=np.array(CHUNK_SEC, dtype=np.float32),
        stride_sec=np.array(STRIDE_SEC, dtype=np.float32),
        source_audio=os.path.basename(path),
    )
    print(f"[ok] {os.path.basename(path)} -> {feats.shape}")

def main():
    files = [p for p in glob.glob(os.path.join(AUDIO_DIR, "**", "*"), recursive=True)
             if os.path.splitext(p)[1].lower() in AUDIO_EXTS]
    files.sort()
    print(f"Found {len(files)} audio files.")
    for p in tqdm(files, desc="HuBERT"):
        try: process_file(p)
        except Exception as e: print(f"[err] {os.path.basename(p)}: {e}")

if __name__ == "__main__":
    main()


Found 15 audio files.


HuBERT:   7%|▋         | 1/15 [00:02<00:28,  2.01s/it]

[ok] ID02_pd_2_0_0s.wav -> (9462, 768)


HuBERT:  13%|█▎        | 2/15 [00:03<00:21,  1.65s/it]

[ok] ID04_pd_2_0_1s.wav -> (7340, 768)


...

[ok] ID32_pd_3_1_1s.wav -> (5265, 768)


HuBERT:  93%|█████████▎| 14/15 [00:24<00:02,  2.32s/it]

[ok] ID33_pd_3_2_2s.wav -> (6140, 768)


HuBERT: 100%|██████████| 15/15 [00:25<00:00,  1.69s/it]

[ok] ID34_pd_2_0_0s.wav -> (6092, 768)
